In [14]:
from my_lda_func import *

In [45]:
# Sample data for analysis
d1 = "Java is a language for programming that develops a software for several platforms. A compiled code or bytecode on Java application can run on most of the operating systems including Linux, Mac operating system, and Linux. Most of the syntax of Java is derived from the C++ and C languages."
d2 = "Python supports multiple programming paradigms and comes up with a large standard library, paradigms included are object-oriented, imperative, functional and procedural."
d3 = "Go is typed statically compiled language. It was created by Robert Griesemer, Ken Thompson, and Rob Pike in 2009. This language offers garbage collection, concurrency of CSP-style, memory safety, and structural typing."
d4 = "A young girl when she first visited magical Underland, Alice Kingsleigh (Mia Wasikowska) is now a teenager with no memory of the place -- except in her dreams."
d5 = "Her life takes a turn for the unexpected when, at a garden party for her fiance and herself, she spots a certain white rabbit and tumbles down a hole after him. Reunited with her friends the Mad Hatter (Johnny Depp), the Cheshire Cat and others, Alice learns it is her destiny to end the Red Queen's (Helena Bonham Carter) reign of terror."
docs=[d1,d2,d3,d4,d5]*10
labels=np.array([1,1,1,0,0]*10)
tf_df, id2word = tf(docs)
lil = []
# for row in tf_df.T.values:
for row in tf_df.values:
    lil_sub = []
    for idx, item in enumerate(row):
        if item:
            lil_sub.append((idx, item))
    lil.append(lil_sub)

In [39]:
res_lda1=my_lda_func(corpus=lil, num_topics=2, id2word=id2word, num_words=5,topics_only=False,passes=1)[1]

performing inference on a chunk of 50 documents
0/50 documents converged within 50 iterations
Update topics
topic diff 0.45876706655499866


In [40]:
import lda_class_carol
b=lda_class_carol.LDA2(docs)
res_lda2=b.lda(2,5,conv_threshold=1e-2,max_iter=100,niter=10)
res_lda2=np.array(b.gamma)

/home/jovyan/work/lda_project/lda_class_carol.py:155: UserWarning: Newton-Raphson has not converged. Try more iterations.
  warn('Newton-Raphson has not converged. Try more iterations.')


In [41]:
from plsa import Corpus, Pipeline, Visualize
from plsa.algorithms import PLSA
from plsa.pipeline import DEFAULT_PIPELINE
pipeline = Pipeline(*DEFAULT_PIPELINE)
corpus_plsa=Corpus(docs,pipeline)
plsa=PLSA(corpus_plsa, 2, True)
result = plsa.fit()
result = plsa.best_of(5)
res_plsa=result.topic_given_doc

In [58]:
# ngram
# see https://scikit-learn.org/stable/modules/feature_extraction.html
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(docs)
analyze = vectorizer.build_analyzer()
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()
res_bigram=bigram_vectorizer.fit_transform(docs).toarray()

(50, 321)

In [42]:
#SVM

In [43]:
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [63]:
methods='plsa,lda1,lda2,bigram'.split(',')
for i,x in enumerate([res_plsa,res_lda1,res_lda2,res_bigram]):
    X_train, X_test, y_train, y_test = train_test_split(x, labels, test_size = 0.2, random_state=0)
    clf = svm.SVC(kernel='linear')
    clf.fit(X_train, y_train)
    clf_predictions = clf.predict(X_test)
    print("Accuracy of {}: {}%".format(methods[i],clf.score(X_test, y_test) * 100 ))

Accuracy of plsa: 100.0%
Accuracy of lda1: 90.0%
Accuracy of lda2: 50.0%
Accuracy of bigram: 100.0%
